In [42]:
import chess
import chess.engine
import chess.pgn

In [43]:
stockfish_path = "D:/Program Files/Stockfish/stockfish/stockfish-windows-x86-64-avx2.exe"

In [44]:
engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

def get_score(board: chess.Board):
    result = engine.analyse(board,  chess.engine.Limit(depth=8))
    value = result['score'].pov(color=chess.WHITE).score(mate_score=900) / 100
    return value

In [45]:
def get_top_n(n: int, board: chess.Board, reverse=False, offset=None):
    move_evaluations = []  
    for move in board.legal_moves:
        board.push(move)
        score = get_score(board)
        move_evaluations.append((move, score))
        board.pop()
    if offset:
        move_evaluations = [(e[0], abs(e[1]-offset)) for e in move_evaluations]
    move_evaluations.sort(key=lambda x: x[1])
    if reverse:
        return [e[0] for e in move_evaluations[:n]]
    return [e[0] for e in move_evaluations[-n:]]

In [46]:
def create_games(n, board: chess.Board, depth, games: set, moves: list, reversed=False):
    if depth == 0:
        if len(games) >= 50:
            return
        if board.fen() not in games:
            games[board.fen()] = moves
        return
    top_moves = get_top_n(n, board, reversed)
    for move in top_moves:
        board_copy = board.copy()
        board_copy.push(move)
        moves_copy = moves.copy()
        moves_copy.append(move)
        create_games(n, board_copy, depth-1, games, moves_copy, not reversed)


In [39]:
def save_offset_games(
    games: list,
    offset = -2,
    additional_moves = 4,
    additional_biased = 2,
):

    with open(f'games/bias-openings_{offset}.pgn', 'w') as pgn_file:
            for game_moves in games:
                board = chess.Board()
                game = chess.pgn.Game()
                node = game
                
                for move in game_moves:
                    board.push(move)
                    node = node.add_variation(move)

                reverse = False
                for _ in range(additional_moves):
                    move = get_top_n(1, board, reverse=reverse)[0]
                    board.push(move)
                    node = node.add_variation(move)
                    reverse = not reverse

                for _ in range(additional_biased):
                    move = get_top_n(1, board, reverse=True, offset=offset)[0]
                    board.push(move)
                    node = node.add_variation(move)

                game.headers["Event"] = "Example Event"
                game.headers["White"] = "Player 1"
                game.headers["Black"] = "Player 2"
                game.headers["Result"] = board.result()

                pgn_file.write(str(game) + "\n\n")

In [36]:
board = chess.Board()
game = chess.pgn.Game()
moves = list()

games = dict()
create_games(3, board, 4, games, moves)


In [41]:
save_offset_games(games.values(), offset=-2)
print('created offset -2')
save_offset_games(games.values(), offset=-3)
print('created offset -3')
save_offset_games(games.values(), offset=2)
print('created offset -2')
save_offset_games(games.values(), offset=3)
print('created offset -3')
save_offset_games(games.values(), offset=-4)
print('created offset -4')
save_offset_games(games.values(), offset=4)
print('created offset -4')

created offset -2
created offset -3
created offset -2
created offset -3
created offset -4
created offset -4
